In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

from env import user, password, host
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
    #import to be able to do decision tress
from sklearn.neighbors import KNeighborsClassifier
    #import to be able to do KNN
from sklearn.linear_model import LogisticRegression
    #import to be able to do logistic regression
from sklearn.ensemble import RandomForestClassifier
    #import to be able to do random forest
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier

import nltk
import unicodedata
import re

In [2]:
def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'


url = get_db_url("spam_db")
sql = "SELECT * FROM spam"

df = pd.read_sql(sql, url, index_col="id")
df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [4]:
def show_counts_and_ratios(df, column):
    labels = pd.concat([df.label.value_counts(),
                    df.label.value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    return labels

show_counts_and_ratios(df, 'label')

,n,percent
ham,4825,0.865937
spam,747,0.134063


In [5]:
# We'll use this split function later to create in-sample and out-of-sample datasets for modeling
def split(df, stratify_by=None):
    """
    3 way split for train, validate, and test datasets
    To stratify, send in a column name
    """
    
    train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[stratify_by])
    
    train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train[stratify_by])
    
    return train, validate, test

In [6]:
df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
train, validate, test = split(df, 'label')

In [8]:
train.head()

,label,text
id,,
236,ham,Or ill be a little closer like at the bus stop...
3851,ham,"No, its true..k,Do u knw dis no. &lt;#&gt; ?"
3906,ham,No that just means you have a fat head
3365,ham,I am waiting for your call sir.
2853,ham,Haha... Hope Ì_ can hear the receipt sound... ...


In [9]:
# Setup our X variables
X_train = train.text
X_validate = validate.text
X_test = test.text

# setup the y variables
y_train = train.label
y_validate= validate.label
y_test = test.label

In [10]:
X_validate

id
1081                    Can u get pic msgs to your phone?
3560    Text BANNEDUK to 89555 to see! cost 150p texto...
3406    Beautiful Truth against Gravity.. Read careful...
4348         ÌÏ bot notes oredi... Cos i juz rem i got...
3592                               Sorry, I'll call later
                              ...                        
4909    WE REGRET TO INFORM U THAT THE NHS HAS MADE A ...
713     Save yourself the stress. If the person has a ...
4332              Y bishan lei... I tot Ì_ say lavender? 
2181                                                  Ok.
3219    Its like that hotel dusk game i think. You sol...
Name: text, Length: 1338, dtype: object

In [11]:
# create the vectorized object
tfidf = TfidfVectorizer()

In [12]:
# fir the vectorized object onto train
tfidf.fit(X_train)

TfidfVectorizer()

In [13]:
# set up on the training data
X_train_vectorized = tfidf.transform(X_train)
X_validate_vectorized = tfidf.transform(X_validate)
X_test_vectorized = tfidf.transform(X_test)

In [14]:
X_train_vectorized.todense().shape

(3119, 6277)

In [15]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))

# Logistic Regression Model

In [16]:
lm = LogisticRegression(random_state=123)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
lm.fit(X_train_vectorized, y_train)

LogisticRegression(random_state=123)

In [17]:
# make your predictions
train['predicted'] = lm.predict(X_train_vectorized)
validate["predicted"] = lm.predict(X_validate_vectorized)
test['predicted'] = lm.predict(X_test_vectorized)

In [18]:
# Train Accuracy
(train.actual == train.predicted).mean()

0.9695415197178583

In [19]:
# Validate Accuracy
    # AKA Out of Sample Accuracy
(validate.actual == validate.predicted).mean()

0.9566517189835575

## Logistic Regression (Class_weight Balanced)

In [20]:
lm2 = LogisticRegression(random_state=123, class_weight='balanced')
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
lm2.fit(X_train_vectorized, y_train)

LogisticRegression(class_weight='balanced', random_state=123)

In [21]:
# make your predictions
train['predicted'] = lm2.predict(X_train_vectorized)
validate["predicted"] = lm2.predict(X_validate_vectorized)
test['predicted'] = lm2.predict(X_test_vectorized)

## KNN Model

In [22]:
knn = KNeighborsClassifier()
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
knn.fit(X_train_vectorized, y_train)

KNeighborsClassifier()

In [23]:
# make your predictions
train['predicted'] = knn.predict(X_train_vectorized)
validate["predicted"] = knn.predict(X_validate_vectorized)
test['predicted'] = knn.predict(X_test_vectorized)

## Decision Tree

In [24]:
clf1 = DecisionTreeClassifier(max_depth=3)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
clf1.fit(X_train_vectorized, y_train)

DecisionTreeClassifier(max_depth=3)

In [25]:
# make your predictions
train['predicted'] = clf1.predict(X_train_vectorized)
validate["predicted"] = clf1.predict(X_validate_vectorized)
test['predicted'] = clf1.predict(X_test_vectorized)

## Ridge Classifier Model

In [26]:
clf2 = RidgeClassifier(random_state=123)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
clf2.fit(X_train_vectorized, y_train)

RidgeClassifier(random_state=123)

In [27]:
# make your predictions
train['predicted'] = clf2.predict(X_train_vectorized)
validate["predicted"] = clf2.predict(X_validate_vectorized)
test['predicted'] = clf2.predict(X_test_vectorized)

## SGDClassifier Model

In [28]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
clf3.fit(X_train_vectorized, y_train)

SGDClassifier(random_state=123)

In [29]:
# make your predictions
train['predicted'] = clf3.predict(X_train_vectorized)
validate["predicted"] = clf3.predict(X_validate_vectorized)
test['predicted'] = clf3.predict(X_test_vectorized)

# Models Performance

In [30]:
X_train.sample()

id
4965    URGENT! We are trying to contact U. Todays dra...
Name: text, dtype: object

In [31]:
text = pd.Series([
    "can u get pic msgs to your phone", 
    "sorry, ill call later", 
    "oh yeah clearly its my fault", 
    "then ur sis how?", 
    "come back to tampa fffuuuuuuu"
])

In [32]:
# Clean our inputs and lemmatize
lines = text.apply(clean)


In [33]:
# we HAVE TO vectorize these inputs if we are going to be able to use the classificaton model
text = tfidf.transform(text)

## Logistic Regression Model Performance

In [34]:
lm.predict(text)

array(['ham', 'ham', 'ham', 'ham', 'ham'], dtype=object)

In [35]:
accuracy = lm.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 0.97


In [36]:
accuracy = lm.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.96


## Logistic Regression Balanced Model Performance

In [37]:
lm2.predict(text)

array(['spam', 'ham', 'ham', 'ham', 'ham'], dtype=object)

In [38]:
accuracy = lm2.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 0.99


In [39]:
accuracy = lm2.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.98


## KNN Performance

In [40]:
accuracy = lm2.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 0.99


In [41]:
accuracy = knn.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.96


## Ridge Classifier Performance

In [42]:
accuracy = clf1.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 0.95


In [43]:
accuracy = clf1.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.94


## Ridge Classifier Performance

In [44]:
accuracy = clf2.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 1.0


In [45]:
accuracy = clf2.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.98


## SGDClassifier Performance

In [46]:
accuracy = clf3.score(X_train_vectorized, y_train)
print(f'Accuracy of Logistic Regression classifier on training set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on training set: 1.0


In [47]:
accuracy = clf3.score(X_validate_vectorized, y_validate)
print(f'Accuracy of Logistic Regression classifier on validate set: {accuracy:.2}')

Accuracy of Logistic Regression classifier on validate set: 0.98
